In [ ]:
# %pip install -q 'bitsandbytes==0.39.1'
# %pip install -q peft
# %pip install -q torchkeras

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from argparse import Namespace
cfg = Namespace()

# model
cfg.model_name_or_path = '/home/huang/models/chatglm2-6b'
cfg.quantization_bit = None

In [ ]:
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_doble_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

tokenizer = AutoTokenizer.from_pretrained(cfg.model_name_or_path, trust_remote_code=True)
model = AutoModel.from_pretrained(cfg.model_name_or_path, quantization_config=bnb_config,
                                  trust_remote_code=True)

In [ ]:
#generate 文字接龙接口
text = '世界上最高的山峰是什么？'
inputs = tokenizer(text)
inputs = {k:torch.tensor([v]) for k,v in inputs.items()}
outputs = model.generate(**inputs,max_new_tokens=64,repetition_penalty=1.1)
tokenizer.batch_decode(outputs) 

In [ ]:
#chat 聊天接口
response,history= model.chat(tokenizer,query='世界上最高的山峰是什么？',history=[])
print(response)

In [ ]:
#stream_chat 流聊天接口(打字机风格)
result = model.stream_chat(tokenizer,query='世界上最高的山峰是什么？',history=[])
for response,history in result:
    print(response,end='\r')

In [ ]:
#注册魔法命令便于jupyter中使用
from torchkeras.chat import ChatGLM 
chatglm = ChatGLM(model,tokenizer,stream=True)

In [ ]:
%%chatglm
你好呀，请介绍一下你自己？

In [ ]:
%%chatglm
你听说过梦中情炉吗？